In [1]:
import scipy.io as io
import numpy as np

In [2]:
dataset = io.loadmat('PaviaU.mat')['paviaU']
gt_labels = io.loadmat('PaviaU_gt.mat')['paviaU_gt']
print(dataset[0][0])
print(gt_labels[0][0])

[ 647  499  464  371  291  319  365  322  296  305  277  219  219  222
  201  162  157  183  204  194  198  216  249  284  294  322  338  342
  336  342  362  365  348  341  324  316  293  274  251  244  263  248
  236  254  255  240  223  203  202  191  185  200  203  181  168  174
  185  187  165  151  144  140  149  169  192  227  287  376  493  633
  763  913 1149 1442 1759 2102 2425 2689 2895 3058 3196 3252 3195 3297
 3542 3550 3537 3545 3514 3477 3468 3433 3408 3420 3416 3335 3256 3226
 3205 3210 3221 3238 3250]
0


In [3]:
a = {x:[] for x in range(1,10)}
(x,y,z) = dataset.shape
for i in range(x):
    for j in range(y):
        l = gt_labels[i][j]
        if l!=0:
            a[l].append(np.array(dataset[i][j]))

In [4]:
for i in range(1,10):
    print(len(a[i]))

6631
18649
2099
3064
1345
5029
1330
3682
947


In [5]:
y = []
X = []
for i in range(1,10):
    for j in range(len(a[i])):
        X.append(a[i][j])
        y.append(i)
X = np.array(X)
y = np.array(y)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils import to_categorical
X = normalize(X,axis=1,return_norm=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test = X_test.reshape(len(X_test),103,1,1)
X_train = X_train.reshape(len(X_train),103,1,1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train = y_train[:,1:]
y_test = y_test[:,1:]

Using TensorFlow backend.


In [22]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten, MaxPooling2D , Dropout
from keras.initializers import RandomUniform
from keras.callbacks import ModelCheckpoint

In [20]:
from keras.initializers import RandomUniform
initializer = RandomUniform(minval=-0.05, maxval=0.05)
model = Sequential()
model.add(Conv2D(20, kernel_size=(11,1), activation='tanh', input_shape=(103,1,1),kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D(pool_size = (3,1)))
model.add(Flatten())
model.add(Dense(100,activation='tanh',kernel_initializer=initializer))
model.add(Dense(9,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 93, 1, 20)         240       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 1, 20)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 620)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               62100     
_________________________________________________________________
dense_10 (Dense)             (None, 9)                 909       
Total params: 63,249
Trainable params: 63,249
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [65]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 50,verbose=1,batch_size=20,callbacks=callbacks_list)

Train on 28659 samples, validate on 14117 samples
Epoch 1/50
28659/28659 [==============================] - 5s 184us/step - loss: 0.1978 - acc: 0.9247 - val_loss: 0.2184 - val_acc: 0.9175

Epoch 00001: val_acc did not improve from 0.92576
Epoch 2/50
28659/28659 [==============================] - 5s 189us/step - loss: 0.1969 - acc: 0.9253 - val_loss: 0.2084 - val_acc: 0.9238

Epoch 00002: val_acc did not improve from 0.92576
Epoch 3/50
28659/28659 [==============================] - 5s 190us/step - loss: 0.1931 - acc: 0.9274 - val_loss: 0.2003 - val_acc: 0.9255

Epoch 00003: val_acc did not improve from 0.92576
Epoch 4/50
28659/28659 [==============================] - 6s 196us/step - loss: 0.1947 - acc: 0.9249 - val_loss: 0.2215 - val_acc: 0.9145

Epoch 00004: val_acc did not improve from 0.92576
Epoch 5/50
28659/28659 [==============================] - 6s 192us/step - loss: 0.1947 - acc: 0.9262 - val_loss: 0.2024 - val_acc: 0.9258

Epoch 00005: val_acc did not improve from 0.92576
Epoch

28659/28659 [==============================] - 6s 192us/step - loss: 0.1535 - acc: 0.9409 - val_loss: 0.1743 - val_acc: 0.9362

Epoch 00041: val_acc did not improve from 0.93674
Epoch 42/50
28659/28659 [==============================] - 5s 191us/step - loss: 0.1510 - acc: 0.9420 - val_loss: 0.1934 - val_acc: 0.9265

Epoch 00042: val_acc did not improve from 0.93674
Epoch 43/50
28659/28659 [==============================] - 6s 200us/step - loss: 0.1511 - acc: 0.9402 - val_loss: 0.1721 - val_acc: 0.9369

Epoch 00043: val_acc improved from 0.93674 to 0.93688, saving model to weights-improvement-43-0.94.hdf5
Epoch 44/50
28659/28659 [==============================] - 6s 205us/step - loss: 0.1494 - acc: 0.9426 - val_loss: 0.1778 - val_acc: 0.9336

Epoch 00044: val_acc did not improve from 0.93688
Epoch 45/50
28659/28659 [==============================] - 6s 204us/step - loss: 0.1490 - acc: 0.9439 - val_loss: 0.1712 - val_acc: 0.9360

Epoch 00045: val_acc did not improve from 0.93688
Epoch 46

In [41]:
indian_pines = io.loadmat('Indian_pines.mat')['indian_pines']

In [42]:
indian_pines.shape

(145, 145, 220)

In [43]:
gt_labels_pines = io.loadmat('Indian_pines_gt.mat')['indian_pines_gt']

In [63]:
len(gt_labels_pines[gt_labels_pines == 14])

1265